In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
ls ../input

In [3]:
import os
import datetime as dt
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 99

import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
sns.set_palette(sns.color_palette('tab20', 20))
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [5]:
competition_data_dir = '../input/data-science-for-good-kiva-crowdfunding/'
additional_data_dir = '../input/additional-kiva-snapshot/'

In [6]:
loans = pd.read_csv(additional_data_dir + 'loans.csv')
kiva_loans = pd.read_csv(competition_data_dir + 'kiva_loans.csv')

In [7]:
merged_loans = pd.merge(kiva_loans, loans, how='left', left_on='id', right_on='loan_id')

In [8]:
loans_lenders = pd.read_csv(additional_data_dir + 'loans_lenders.csv')
country_stats = pd.read_csv(additional_data_dir + 'country_stats.csv')

In [9]:
loans.groupby(['country_code'])

In [10]:
loans.country_code.value_counts().head(20).plot.bar()

In [11]:
lenders = pd.read_csv(additional_data_dir + 'lenders.csv')

In [12]:
lenders.country_code.value_counts().head(20).plot.bar()

In [13]:
loan_country_cnt = loans.groupby(['country_code']).count()[['loan_id']].reset_index()

In [14]:
loan_country_cnt.columns = ['country_code', 'loan_cnt']
loan_country_cnt = loan_country_cnt.sort_values(by='loan_cnt', ascending=False)

In [15]:
lender_country_cnt = lenders.groupby(['country_code']).count()[['permanent_name']].reset_index()
lender_country_cnt.columns = ['country_code', 'lender_cnt']
lender_country_cnt = lender_country_cnt.sort_values(by='lender_cnt', ascending=False)

In [16]:
country_count = pd.merge(loan_country_cnt, lender_country_cnt, how='outer', on='country_code')
country_count = country_count.merge(country_stats[['country_code']])
threshold = 10
country_count.loc[country_count.loan_cnt < threshold, 'loan_cnt'] = 0
country_count.loc[country_count.lender_cnt < threshold, 'lender_cnt'] = 0
country_count = country_count.fillna(0)

In [17]:
country_count.head()

In [19]:
country_stats = pd.merge(country_count, country_stats, how='inner', on='country_code')

In [20]:
country_stats['population_in_poverty'] = country_stats['population'] * country_stats['population_below_poverty_line'] / 100.

In [21]:
data = [dict(
        type='choropleth',
        locations=country_stats['country_name'],
        locationmode='country names',
        z=np.log10(country_stats['loan_cnt'] + 1),
        text=country_stats['country_name'],
        colorscale='Reds',
        reversescale=False,
        marker=dict(line=dict(color='rgb(180,180,180)', width=0.5)),
        colorbar=dict(autotick=False, tickprefix='', title='Loans'),
)]

In [22]:
layout = dict(
    title = 'Number of loans by Country',
    geo = dict(showframe=False, showcoastlines=True, projection=dict(type='Mercator'))
)

In [25]:
fig = dict(data=data, layout=layout)

In [26]:
py.iplot(fig, validate=False, filename='loans-world-map')

In [27]:
country_stats[country_stats.country_code=='US'].loan_cnt

In [28]:
np.log10(country_stats[country_stats.country_code=='US'].loan_cnt + 1)

In [29]:
data = [dict(
        type='choropleth',
        locations=country_stats['country_name'],
        locationmode='country names',
        z=np.log10(country_stats['lender_cnt'] + 1),
        text=country_stats['country_name'],
        colorscale='Blues',
        reversescale=True,
        marker=dict(line=dict(color='rgb(180,180,180)', width=0.5)),
        colorbar=dict(autotick=False, tickprefix='', title='Lenders'),
)]
layout = dict(
    title = 'Number of lenders by Country',
    geo = dict(showframe=False, showcoastlines=True, projection=dict(type='Mercator'))
)
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='lenders-world-map')

In [30]:
data = [dict(
        type='choropleth',
        locations=country_stats['country_name'],
        locationmode='country names',
        z=(country_stats['lender_cnt'] / country_stats['population']),
        text=country_stats['country_name'],
        colorscale='Blues',
        reversescale=True,
        marker=dict(line=dict(color='rgb(180,180,180)', width=0.5)),
        colorbar=dict(autotick=False, tickprefix='', title='Lenders'),
)]
layout = dict(
    title = 'Percentage of lenders by Country',
    geo = dict(showframe=False, showcoastlines=True, projection=dict(type='Mercator'))
)
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='lenders-world-map')

In [31]:
data = [dict(
        type='choropleth',
        locations=country_stats['country_name'],
        locationmode='country names',
        z=(country_stats['life_expectancy']),
        text=country_stats['country_name'],
        colorscale='Viridis',
        reversescale=True,
        marker=dict(line=dict(color='rgb(180,180,180)', width=0.5)),
        colorbar=dict(autotick=False, tickprefix='', title='Age'),
)]
layout = dict(
    title = 'Life Expectancy by Country',
    geo = dict(showframe=False, showcoastlines=True, projection=dict(type='Mercator'))
)
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='lenders-world-map')

In [32]:
loan_coords = pd.read_csv(additional_data_dir + 'loan_coords.csv')

In [33]:
loans_with_coords = loans[['loan_id', 'country_name', 'town_name']].merge(loan_coords, how='left', on='loan_id')

In [34]:
town_counts = loans_with_coords.groupby(['country_name', 'town_name', 'latitude', 'longitude']).count()[['loan_id']].reset_index()
town_counts.head()

In [35]:
town_counts = town_counts.rename(columns={'loan_id': 'loan_cnt'})

In [36]:
town_counts = town_counts.sort_values(by='loan_cnt', ascending=False)

In [37]:
data = [dict(
    type='scattergeo',
    lon = town_counts['longitude'],
    lat = town_counts['latitude'],
    text = town_counts['town_name'],
    mode = 'markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size= 5 * (np.log10(town_counts.loan_cnt + 1) - 1),
        color = np.log10(town_counts['loan_cnt'] + 1),
        opacity = 0.7,
        line = dict(width=0),
        colorscale='Reds',
        reversescale=False,
        showscale=True
    ),
)]

In [38]:
layout = dict(
    title = 'Number of Loans by Region (Enhanced)',
    hovermode='closest',
    geo = dict(showframe=False, showland=True, showcoastlines=True, showcountries=True,
               countrywidth=1, projection=dict(type='Mercator'))
)

In [39]:
fig = go.Figure(data=data, layout=layout)

In [40]:
py.iplot(fig, filename='all-town-loans')

In [41]:
sns.lmplot('population_in_poverty', 'life_expectancy', data=country_stats)

In [42]:
country_stats.iloc[np.argmax(country_stats.population_in_poverty), :].kiva_country_name

In [43]:
sns.regplot(np.log(country_stats['population_in_poverty'] + 1), country_stats['life_expectancy'])

In [44]:
sns.regplot(np.log(country_stats['population_in_poverty'] + 1), country_stats['expected_years_of_schooling'])

In [45]:
sns.regplot(np.log(country_stats['population'] + 1), country_stats['expected_years_of_schooling'])

In [46]:
country_stats.groupby('continent').expected_years_of_schooling.mean().plot.bar();

In [47]:
sns.boxplot(x='continent', y='expected_years_of_schooling', data=country_stats)

In [48]:
sns.violinplot(x='continent', y='life_expectancy', data=country_stats)

In [49]:
country_stats[country_stats.continent=='Asia'].life_expectancy.plot.hist(bins=8)

In [50]:
sns.swarmplot(x='continent', y='life_expectancy', data=country_stats)

In [51]:
sns.regplot(np.log(country_stats['population_in_poverty'] + 1), country_stats['loan_cnt'])

In [52]:
sns.regplot(country_stats['population_in_poverty'], country_stats['loan_cnt'])

In [55]:
q99 = country_stats.loan_cnt.quantile(0.99)

In [56]:
q1 = country_stats.loan_cnt.quantile(0.01)

In [57]:
country_stats.loc[country_stats.loan_cnt > q99, 'loan_cnt'] = q99

In [58]:
sns.regplot(np.log(country_stats['population_in_poverty']+1), country_stats['loan_cnt'])

In [59]:
country_stats['population_in_poverty'].plot.hist(bins=20);

In [60]:
q99 = country_stats.population_in_poverty.quantile(0.99)

In [61]:
country_stats.loc[country_stats.population_in_poverty > q99, 'population_in_poverty'] = q99

Let's look at the distribution again:

In [63]:
country_stats['population_in_poverty'].plot.hist(bins=20);

In [64]:
sns.regplot(country_stats['population_in_poverty'],country_stats['loan_cnt'])

In [65]:
sns.regplot(country_stats['population_below_poverty_line'],country_stats['loan_cnt'])